In [62]:
# Definition of a job
class Job:
    
    f: int # family
    w: int # weight
    d: int # due date
    C: int = None # completion time
    
    def __init__(self, family: int, weight: int, due_date: int):
        self.f = family
        self.w = weight
        self.d = due_date
        
    def __str__(self) -> str:
        return f"Job(f: {self.f}, w: {self.w}, d: {self.d}, C: {self.C})"
    
    def __repr__(self) -> str:
        return str(self)

# Function to get the family of a job
def s(j: Job) -> int:
    return j.f

import random

t = 0

BATCH_SIZE = 5
MIN_FAMILIES = 5
MAX_FAMILIES = 25
MIN_PROCESSING_TIME = 8
MAX_PROCESSING_TIME = 12
MAX_WEIGHT = 10
MIN_JOBS = 100
MAX_JOBS = 1000
MIN_DUE_DATE = 20
MAX_DUE_DATE = 1000
MAX_MACHINES = 10

# Generate random data
familyProcessingTimes = [random.randint(MIN_PROCESSING_TIME, MAX_PROCESSING_TIME) for i in range(random.randint(MIN_FAMILIES, MAX_FAMILIES))]
print("Family processing times:\n" + str(familyProcessingTimes))

machines = [None for i in range(random.randint(1, MAX_MACHINES))]
print("Number of machines:\n" + str(len(machines)))

jobs = [Job(family=random.randint(0, len(familyProcessingTimes) - 1), weight=random.randint(1, MAX_WEIGHT), due_date=random.randint(1, MIN_DUE_DATE)) for i in range(random.randint(MIN_JOBS, MAX_JOBS))]
print("Number of jobs:\n" + str(len(jobs)))

Family processing times:
[9, 8, 11, 12, 10, 11, 8, 8, 8, 11, 8, 8, 8, 10, 9, 10, 11, 8, 10, 12, 10, 11, 12, 12]
Number of machines:
6
Number of jobs:
777


TWT Funktion: $\sum_{j=1}^n w_j T_j = \sum_{j=1}^n w_j \max\{C_j - d_j; 0\}$

In [63]:
def twt(jobs: list[Job]) -> int:
    return sum(job.w * max(job.C - job.d, 0) for job in jobs)

In [64]:
from math import exp


def atc(job: Job, p_avg: float, kappa: int) -> int:
    return (job.w / familyProcessingTimes[s(job)]) * kappa * exp(-(max(job.d - familyProcessingTimes[s(job)] - t, 0))/(kappa * p_avg))

Eröffnungsheuristik: ATC-BATC-Heuristic

In [65]:
from statistics import mean
import pandas as pd


best_machines = None
best_twt = None
kappa_twt_stats = list()

# iterate over different kappa values
for kappa in [0.5 * l for l in range(1, 10)]:
    # copy machines and jobs for hyperparameter (kappa) search
    temp_machines = machines.copy()
    temp_jobs = jobs.copy()
    # iterate over free machines
    for free_machine_idx in [index for index, machine in enumerate(temp_machines) if machine is None]:
        # break if no jobs are left
        if(len(temp_jobs) == 0):
            break
        # sort all jobs by ATC Index
        p_avg = mean([familyProcessingTimes[s(job)] for job in temp_jobs])
        temp_jobs.sort(
            key=lambda j: atc(j, p_avg, kappa=kappa),
            reverse=True
        )
        # retrieve families that are not currently being processed
        nonidle_families = set([batch["family"] for batch in temp_machines if batch is not None])
        idle_families = set([s(job) for job in temp_jobs]) - nonidle_families
        # construct a batch of the top atc jobs for each idle family and get the BATC Index of each batch
        if len(idle_families) > 0:
            batc_index = list()
            for family in idle_families:
                family_jobs = [job for job in temp_jobs if s(job) == family]
                batch_jobs = family_jobs[:min(BATCH_SIZE, len(family_jobs))]
                batc_index.append({
                    "family": family,
                    "batc_index": sum([atc(job, p_avg, kappa=kappa) for job in batch_jobs]), 
                    "batch_jobs": batch_jobs
                })
            # get the best family batch by argmax batc and add it to the free machine
            best_batch = max(batc_index, key=lambda b: b["batc_index"])
            temp_machines[free_machine_idx] = best_batch
            # assign completion times to the jobs in the batch and remove them from the job list
            for job in best_batch["batch_jobs"]:
                job.C = t + familyProcessingTimes[s(job)]
                temp_jobs.remove(job)
    # get the TWT of the current machine configuration and update the best machine configuration if necessary
    temp_twt = twt([job for batch in temp_machines if batch is not None for job in batch["batch_jobs"]])
    kappa_twt_stats.append({"kappa": kappa, "twt": temp_twt})
    if best_twt is None or temp_twt < best_twt:
        best_machines = temp_machines
        best_twt = temp_twt
    
machines = best_machines
pd.DataFrame(kappa_twt_stats)

,kappa,twt
0,0.5,845
1,1.0,810
2,1.5,810
3,2.0,741
4,2.5,741
5,3.0,732
6,3.5,716
7,4.0,708
8,4.5,708


In [66]:
pd.DataFrame(machines)

,family,batc_index,batch_jobs
0,17,25.000000,"[Job(f: 17, w: 10, d: 4, C: 8), Job(f: 17, w: ..."
1,6,24.000000,"[Job(f: 6, w: 10, d: 1, C: 8), Job(f: 6, w: 10..."
2,11,23.634124,"[Job(f: 11, w: 10, d: 8, C: 8), Job(f: 11, w: ..."
3,8,23.277975,"[Job(f: 8, w: 10, d: 7, C: 8), Job(f: 8, w: 10..."
4,1,22.238924,"[Job(f: 1, w: 10, d: 1, C: 8), Job(f: 1, w: 9,..."
5,7,21.493770,"[Job(f: 7, w: 10, d: 12, C: 8), Job(f: 7, w: 9..."
